# Example usage of Feature Logic package

In [ ]:
%load_ext autoreload
%autoreload 2

from pyspark.sql import functions as sf, DataFrame, SparkSession

from feature_logic.main import compute_features
from feature_logic.feature_groups.zone import Zone

spark = SparkSession.builder.getOrCreate()

# Basic usage

Define the features you want to compute and call the `compute_features` function.

In [2]:
aggregation_level = "avatarId"
scope = spark.read.parquet("data/wow.parquet").select(aggregation_level).distinct()

features = compute_features(
    scope=scope, 
    feature_groups=[
        Zone(spark=spark, features_of_interest=["darkshore_likelyhood"], aggregation_level=aggregation_level),
    ]).toPandas()
print("shape: ", features.shape)
print(features.head())

shape:  (91064, 1)
   avatarId
0        29
1      1806
2      9233
3      2040
4        26


### You can compute features on different aggregation levels

As long as the feature group supports it you can compute features on different aggregation levels.
One call to the feature logic package (with `compute_features`) will always only be on one level since it makes sense to compute your features on a specific level.

See example below.


In [3]:
for aggregation_level in ["avatarId", "guild"]:
    scope = spark.read.parquet("data/wow.parquet").select(aggregation_level).distinct()

    features = compute_features(
        scope=scope, 
        feature_groups=[
            Zone(spark=spark, features_of_interest=["darkshore_likelyhood"], aggregation_level=aggregation_level),
        ]).toPandas()
    print("shape: ", features.shape)
    print(features.head())

shape:  (91064, 1)
   avatarId
0        29
1      1806
2      9233
3      2040
4        26


shape:  (514, 1)
   guild
0     26
1     29
2    474
3     65
4    191
